In [1]:
#https://gymnasium.farama.org/environments/atari/
#pip install gymnasium[atari]
#pip install gymnasium[accept-rom-license]
#https://www.youtube.com/watch?v=hCeJeq8U0lo&t=447s
import gymnasium as gym
import random

In [7]:
env = gym.make('ALE/SpaceInvaders-v5', render_mode="human")
height, width, channels = env.observation_space.shape
actions=env.action_space.np_random

In [3]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [10]:
episodes = 5
for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        observation, reward, terminated, truncated, info = env.step(action)
        score +=reward
        done = terminated or truncated
    print("episode: " + str(episode) + ", score: " + str(score))
env.close()

episode: 0, score: 230.0


: 

: 